In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

#file_p6=np.loadtxt("result.txt", delimiter = ";")
result=np.loadtxt("result.dat")

MJD=np.zeros(len(result)-17)
p6=np.zeros(len(result)-17)
err_y=np.zeros(len(result)-17)


for i in range(len(p6)):
  MJD[i]=result[i+17][0]-result[17][0] #Me quedo con los datos post-glitch
  p6[i]=result[i+17][3] 
  err_y[i]=result[i+17][4]

p6_pre=np.zeros(16)
for i in range(16):
  p6_pre[i]=result[i][3]


In [4]:
F_ref = 1e-11

In [5]:
def eq_3_new(t, t_exp, I_exp_I_c,A):
    return (-I_exp_I_c * (1.381518*1e-5 / F_ref) / (t_exp*24*3600) * np.exp(-t/t_exp)) + A

def eq_9_new(t, t_nl, t_0, I_a_I_c):
    return  I_a_I_c * -1.55645*(1e-11/F_ref) * (1-((1-(t_nl/t_0)*np.log(1+(np.exp(t_0/t_nl)-1)*np.exp(-t/t_nl)))/(1-np.exp(-t/t_nl))))

def eq_11_new(t, t_0, I_a_I_c):
    return I_a_I_c * -1.55645*(1e-11/F_ref) * (1.0-t/t_0) 

def modelo_1(t,t_exp,I_exp_I_c, A,t_nl, t_0, I_a_I_c):
    return eq_3_new(t, t_exp, I_exp_I_c,A) + eq_9_new(t, t_nl, t_0, I_a_I_c)

def modelo_2(t,t_exp,I_exp_I_c, A,t_0, I_a_I_c):
    return eq_3_new(t, t_exp, I_exp_I_c,A) + eq_11_new(t, t_0, I_a_I_c)

#Modelo 1:

p6_new = [p/F_ref for p in p6]
err_y_new = [e/F_ref for e in err_y]

ini_1 = [6,0.01,-1.54,6, 1000, 0.006]
bs_1 = ([1,1e-3,-1.6,1,100,1e-3],[50,1e-1,-1.4,20,2000,1e-1]) # ( min values, max values ) = ( (min_n, ...), (max_n, ...) )
pars_1, cov_1 = curve_fit(f=modelo_1, xdata=MJD, ydata=p6_new, sigma=err_y_new, p0=ini_1,
                        absolute_sigma=True, bounds=bs_1)

ini_2 = [6, 1e-2,-1.54, 1000, 0.006]
bs_2 = ([1, 1e-3, -1.6,100, 1e-3],[50, 1e-1,-1.4, 2000, 1e-1]) # ( min values, max values ) = ( (min_n, ...), (max_n, ...) )
pars_2, cov_2 = curve_fit(f=modelo_2, xdata=MJD, ydata=p6_new, sigma=err_y_new, p0=ini_2,
                        absolute_sigma=True, bounds=bs_2)

lins=np.linspace(0,320,1000)

plt.figure(figsize=(5,5))
plt.errorbar(MJD, p6_new, yerr=err_y_new, label="F1", fmt=".", color="black")
plt.plot(lins, modelo_1(lins,*pars_1), label="Modelo 1", color="red", linestyle="--",linewidth=2)
plt.plot(lins, modelo_2(lins,*pars_2), label="Modelo 2", color="blue", linestyle="--")
plt.grid()
plt.legend()
#plt.savefig("Ajuste")
plt.show()


perr1 = np.sqrt(np.diag(cov_1))
Texp1, Iexp_IC1, A1, Tnl1, t01, Ia_Ic1 = pars_1 # [0], pars_2[1], pars_11[2]

perr2 = np.sqrt(np.diag(cov_2))
Texp2, Iexp_IC2, A2, t02, Ia_Ic2 = pars_2 # [0], pars_2[1], pars_11[2]

print("\nModel 1:")
print(f"{Texp1 = :.2f} +/- {perr1[0]:.2f}")
print(f"{Iexp_IC1 = :.4f} +/- {perr1[1]:.4f}")
print(f"{A1 = :.3f} +/- {perr1[2]:.3f}")
print(f"{Tnl1 = :.2f} +/- {perr1[3]:.2f}")
print(f"{t01 = :.2f} +/- {perr1[4]:.2f}")
print(f"{Ia_Ic1 = :.5f} +/- {perr1[5]:.5f}")

print("\nRelative errors:")
print(f"{perr1[0]/Texp1 = :.2f}")
print(f"{perr1[1]/Iexp_IC1 = :.2f}")
print(f"{perr1[2]/A1 = :.2f}")
print(f"{perr1[3]/Tnl1 = :.2f}")
print(f"{perr1[4]/t01 = :.2f}")
print(f"{perr1[5]/Ia_Ic1 = :.5f}")

print("\nModel 2:")
print(f"{Texp2 = :.2f} +/- {perr2[0]:.2f}")
print(f"{Iexp_IC2 = :.4f} +/- {perr2[1]:.4f}")
print(f"{A2 = :.2f} +/- {perr2[2]:.2f}")
print(f"{t02 = :.2f} +/- {perr2[3]:.2f}")
print(f"{Ia_Ic2 = :.5f} +/- {perr2[4]:.5f}")

print("\nRelative errors:")
print(f"{perr2[0]/Texp2 = :.2f}")
print(f"{perr2[1]/Iexp_IC2 = :.2f}")
print(f"{perr2[2]/A2 = :.2f}")
print(f"{perr2[3]/t02 = :.2f}")
print(f"{perr2[4]/Ia_Ic2 = :.5f}")

<ipython-input-5-522065f010a2>:5: RuntimeWarning: invalid value encountered in true_divide
  return  I_a_I_c * -1.55645*(1e-11/F_ref) * (1-((1-(t_nl/t_0)*np.log(1+(np.exp(t_0/t_nl)-1)*np.exp(-t/t_nl)))/(1-np.exp(-t/t_nl))))


ValueError: ignored